In [19]:
from graph import graph_feature
from text import text_feature
import numpy as np
from scipy.sparse import hstack
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
import sklearn
import xgboost as xgb

In [2]:
t = text_feature()
g = graph_feature()

X_train_graph = g[0]
X_train_text = t[0]
y_train = g[1]
X_test_graph = g[2]
X_test_text = t[2]

     id  year                                              title  \
0  1001  2000              compactification geometry and duality   
1  1002  2000  domain walls and massive gauged supergravity p...   
2  1003  2000     comment on metric fluctuations in brane worlds   
3  1004  2000         moving mirrors and thermodynamic paradoxes   
4  1005  2000  bundles of chiral blocks and boundary conditio...   

                       authors  \
0            Paul S. Aspinwall   
1  M. Cvetic, H. Lu, C.N. Pope   
2     Y.S. Myung, Gungwon Kang   
3               Adam D. Helfer   
4      J. Fuchs, C. Schweigert   

                                            abstract  
0  these are notes based on lectures given at tas...  
1  we point out that massive gauged supergravity ...  
2  recently ivanov and volovich hep-th 9912242 cl...  
3  quantum fields responding to moving mirrors ha...  
4  proceedings of lie iii clausthal july 1999 var...  
Number of classes: 28
Train matrix dimensionality: (1534

In [5]:
print X_train_graph.shape
print X_train_text.shape

(15341, 32)
(15341, 158148)


In [10]:
X_train = hstack((X_train_graph, X_train_text))
X_test = hstack((X_test_graph, X_test_text))

# Cross validation

In [13]:
clf = LogisticRegression()
scores = cross_val_score(clf, X_train, y_train, cv=3, scoring='neg_log_loss')
print("scores", scores)

('scores', array([-2.10494829, -2.13434191, -2.10656815]))


In [ ]:
clf1 = LogisticRegression(random_state=1)
clf2 = xgb.XGBClassifier()
eclf = VotingClassifier(estimators=[('lr', clf1), ('xgb', clf2)], voting='hard')

for clf, label in zip([clf1, clf2, eclf], ['Logistic Regression', 'Xgboost', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train, cv=3, scoring='neg_log_loss')
    print("Log loss: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Log loss: -2.12 (+/- 0.01) [Logistic Regression]
